### Parameters to Update
**CapacityMetricsWorkspace**: The name of the workspace where the capacity metrics app exists.

**CapacityMetricsDataset**: The name of the capacity metrics app dataset.

In [ ]:
CapacityMetricsWorkspace = 'Microsoft Fabric Capacity Metrics'
CapacityMetricsDataset = 'Fabric Capacity Metrics'

#### Get the workspace id

In [ ]:
workspaceId = spark.conf.get('trident.workspace.id')
print(f"{workspaceId=}")

In [ ]:
import requests, json

header = {'Authorization': f'Bearer {mssparkutils.credentials.getToken("pbi")}'
          ,"Content-Type": "application/json"
          }

response = requests.request(method='get', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}', headers=header)
workspaceName = response.json().get('displayName')
print(f'{workspaceName=}')

#### Create lakehouses

In [ ]:
from notebookutils import mssparkutils

for lakehouseName in ['LH_SampleData', 'LH_QueryResults']:
    try:
        lakehouseItem = mssparkutils.lakehouse.create(lakehouseName, "", workspaceId)
    except Exception as e:
        # print(e)
        pass

LH_SampleData = mssparkutils.lakehouse.get("LH_SampleData", workspaceId)
LH_QueryResults = mssparkutils.lakehouse.get("LH_QueryResults", workspaceId)
print(f'{LH_SampleData=}')
print(f'{LH_QueryResults=}')

#### Get notebook definition from github

In [ ]:
import requests, json, base64
url = "https://raw.githubusercontent.com/bretamyers/FabricTools/main/FabricDWQueryCostAnalyzer/src/NB_Query_Cost_Analyzer.ipynb"
response = requests.get(url)

b = base64.b64encode(bytes(json.dumps(response.json()), 'utf-8')) # convert to bytes
base64_str = b.decode('utf-8') # convert bytes to string
notebookDefDict = json.loads(base64.b64decode(base64_str).decode('utf-8')) # convert base64 bytes to string and then to dictionary
notebookDefDict['metadata']['dependencies']['lakehouse']['default_lakehouse'] = LH_QueryResults.get('id')
notebookDefDict['metadata']['dependencies']['lakehouse']['default_lakehouse_name'] = LH_QueryResults.get('displayName')
notebookDefDict['metadata']['dependencies']['lakehouse']['default_lakehouse_workspace_id'] = LH_QueryResults.get('workspaceId')

for cell in notebookDefDict['cells']:
    if cell['id'] == "20be0399-9430-4b52-a3ab-1c911fc0d69a":
        cell['source'] = [
                f"FabricDWWorkspaceName = '{workspaceName}'\n",
                "FabricDWName = 'WH_SampleData'\n",
                "ConcurrencyNum = 1 # The number of workers that will be executing queries at once. Every worker will execute each query defined in the query list.\n",
                f"CapacityMetricsWorkspace = '{CapacityMetricsWorkspace}'\n",
                f"CapacityMetricsDataset = '{CapacityMetricsDataset}'\n",
                f"QueryRepeatCount = 4 # Number of times a query will run eg. queryRepeatCount = 4, queryList = [query1, query2] will become [query1, query1, query1, query1, query2, query2, query2]\n"
                f"StoreQueryResults = False\n",
                f"RunName = '' # The name of the run. If not specified, one will be generated with the following format. 'Run_{{yyyyMMdd}}_{{hhmmss}}'",
                f"QueryList = '' # A list of queries to be executed. Eg. \"['SELECT COUNT(*) FROM tblA', 'SELECT 1 AS a']\"",
            ]
notebookDefBase64 = base64.b64encode(json.dumps(notebookDefDict).encode('utf-8')).decode('utf-8') # convert dictionary back to base64 string

#### Create or update notebook from the definition

In [ ]:
import requests, json, time, base64

header = {'Authorization': f'Bearer {mssparkutils.credentials.getToken("pbi")}'
          ,"Content-Type": "application/json"
          }

response = requests.request(method='get', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/notebooks', headers=header)

notebookFound = False
if response.status_code == 200:
    # Check if notebook already exists
    for notebook in response.json().get('value'):
        if notebook.get('displayName') == 'NB_Query_Cost_Analyzer':
            notebookFound = True
            body = {"definition": {
                "format": "ipynb",
                "parts": [
                    {
                        "path": "notebook-content.py"
                        ,"payload": notebookDefBase64
                        ,"payloadType": "InlineBase64"
                    }
                ]
                }}
            response = requests.request(method='post', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/notebooks/{notebook.get("id")}/updateDefinition',  headers=header, data=json.dumps(body))

            if response.status_code == 202:
                responseStatus = response
                for retry in range(5):
                    time.sleep(int(responseStatus.headers.get('Retry-After')))
                    responseStatus = requests.request(method='get', url=responseStatus.headers.get('Location'), headers=header)
                    if responseStatus.json().get('status') == 'Succeeded':
                        break
                
if not notebookFound:
    print('Not found')
    body = {
        "displayName": "NB_Query_Cost_Analyzer",
        "description": "A notebook to run data warehouse queries and capture the duration and cost of those queries.",
        "definition": {
            "format": "ipynb",
            "parts": [
                {
                    "path": "notebook-content.py"
                    ,"payload": notebookDefBase64
                    ,"payloadType": "InlineBase64"
                }
            ]
        }
    }

    response = requests.request(method='post', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/notebooks', headers=header, data=json.dumps(body))

    if response.status_code == 202:
        time.sleep(int(response.headers.get('Retry-After')))
        for retry in range(5):
            responseStatus = requests.request(method='get', url=response.headers.get('Location'), headers=header)
            print(responseStatus.text)
            print(responseStatus.headers)
            if responseStatus.headers.get('Location') is not None:
                break
            else:
                time.sleep(int(responseStatus.headers.get('Retry-After')))
    else:
        print(f"Error - {response.json()}")

#### Get the notebook id

In [ ]:
import requests, json

header = {'Authorization': f'Bearer {mssparkutils.credentials.getToken("pbi")}'
          ,"Content-Type": "application/json"
          }
          
# Need to implement for pagination
response = requests.request(method='get', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/notebooks', headers=header)

for notebook in response.json().get('value'):
    if notebook.get('displayName') == 'NB_Query_Cost_Analyzer':
        notebookId = notebook.get('id')
        print(f'{notebookId=}')


#### Create Warehouse

In [ ]:
import requests, json, time

body = {"displayName": "WH_SampleData"}

response = requests.request(method='post', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/warehouses', headers=header, data=json.dumps(body))

if response.status_code == 202:
    print(f'Creating Warehouse {body.get("displayName")}')
    time.sleep(int(response.headers.get('Retry-After')))
    for retry in range(5):
        responseStatus = requests.request(method='get', url=response.headers.get('Location'), headers=header)
        print(responseStatus.text)
        print(responseStatus.headers)
        if responseStatus.json().get('status') != 'Succeeded':
            time.sleep(int(responseStatus.headers.get('Retry-After')))
        else:
            if responseStatus.headers.get('Location') is None: # no result was return but operation completed
                break
            else:
                responseResult = requests.request(method='get', url=responseStatus.headers.get('Location'), headers=header)
                print(f"Succeeded {responseResult.json()}")
                break
else:
    print(response.text)

In [ ]:
import requests, json, time

response = requests.request(method='get', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/warehouses', headers=header)

for warehouse in response.json().get('value'):
    if warehouse.get('displayName') == 'WH_SampleData':
        WH_SampleData = warehouse
        break
print(WH_SampleData)

#### Load data to Lakehouse

In [ ]:
import azure.storage.blob
import pandas as pd
from notebookutils import mssparkutils

spark.conf.set('spark.sql.parquet.int96RebaseModeInWrite', 'LEGACY')
spark.conf.set('spark.sql.parquet.datetimeRebaseModeInWrite', 'LEGACY')

account_url = "https://fabrictutorialdata.blob.core.windows.net"
blob_service_client = azure.storage.blob.BlobServiceClient(account_url)
container_client = blob_service_client.get_container_client('sampledata')

tableList = list()
for blob in container_client.walk_blobs(name_starts_with='WideWorldImportersDW/parquet/tables/', delimiter='/'):
    tableName = blob.name.split('/')[-1].split('.parquet')[0]
    tableList.append(tableName)

for table in tableList:
    print(f'Loading Table: {table}')
    if mssparkutils.fs.exists(f"{LH_SampleData.get('properties').get('abfsPath')}/Tables/{table}"):
        mssparkutils.fs.rm(f"{LH_SampleData.get('properties').get('abfsPath')}/Tables/{table}", recurse=True)

    for blob in container_client.walk_blobs(name_starts_with=f"WideWorldImportersDW/parquet/tables/", delimiter='/'):
        if f'{table}.parquet' == blob.name.split('/')[-1]:
            dfPandas = pd.read_parquet(f'{account_url}/sampledata/{blob.name}', engine='pyarrow', storage_options={'anon': True})

            if not dfPandas.empty:
                dfSpark = spark.createDataFrame(dfPandas)
                dfSpark.write.format('delta').mode('append').option('mergeSchema', "true").save(f"{LH_SampleData.get('properties').get('abfsPath')}/Tables/{table}")
            else:
                print('Bad file', blob.name)

##### Trigger the sql endpoint metadata sync process

In [ ]:
import json, requests
import time
from notebookutils import mssparkutils

header = {'Authorization': f'Bearer {mssparkutils.credentials.getToken("pbi")}'
          ,"Content-Type": "application/json"
          }

workspaceId = spark.conf.get("trident.workspace.id")

response = requests.request(method='get', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/lakehouses', headers=header)

lakehouseId = [lakehouse.get('id') for lakehouse in response.json().get('value') if lakehouse.get('displayName') == 'LH_SampleData'][0]
sqlendpointId = requests.request(method='get', url=f"https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/lakehouses/{lakehouseId}", headers=header).json()['properties']['sqlEndpointProperties']['id']

payload = {"commands":[{"$type":"MetadataRefreshExternalCommand"}]}
response = requests.request(method='post', url=f"https://api.fabric.microsoft.com/v1.0/myorg/lhdatamarts/{sqlendpointId}", data=json.dumps(payload), headers=header)

batchId = response.json()["batchId"]
progressState = response.json()["progressState"]

iterationCnt = 0
while progressState == 'inProgress':
    time.sleep(5)
    statusresponsedata = requests.request(method='get', url=f"https://api.fabric.microsoft.com/v1.0/myorg/lhdatamarts/{sqlendpointId}/batches/{batchId}", headers=header)
    progressState = statusresponsedata.json()["progressState"]
    print(f"Sync state: {progressState}")
    if iterationCnt > 24:
        break # A sync should not take more than 2 minutes. If it does, then break our the loop

table_details = [
        {
        'tableName': table['tableName'],
         'sqlSyncState':  table['sqlSyncState']
        }
        for table in statusresponsedata.json()['operationInformation'][0]['progressDetail']['tablesSyncStatus']
    ]

print(json.dumps(table_details, indent=2))

#### Load data to Warehouse

In [ ]:
from notebookutils import mssparkutils  
from pyspark.sql import functions as F
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType, TimestampType, DoubleType
import pyodbc, struct, itertools, time, datetime, re, uuid, json

connectionString = f'DRIVER={{ODBC Driver 18 for SQL Server}};SERVER={WH_SampleData.get("properties").get("connectionString")};Database={WH_SampleData.get("displayName")}'

# Use the credentials of the user executing the notebook
token = bytes(mssparkutils.credentials.getToken('pbi'), "UTF-8")
encoded_bytes = bytes(itertools.chain.from_iterable(zip(token, itertools.repeat(0))))
tokenstruct = struct.pack("<i", len(encoded_bytes)) + encoded_bytes

def get_result_set(cursor):
    if cursor.description:
        resultList = cursor.fetchall()
        resultColumns = columns = [column[0] for column in cursor.description]
    else:
        resultList = []
        resultColumns = []
    return [dict(zip(resultColumns, [str(col) for col in row])) for row in resultList]

for attempt in range(20):
    try:
        with pyodbc.connect(connectionString, attrs_before = { 1256:tokenstruct }) as conn:
            with conn.cursor() as cursor:
                for retry in range(5):
                    cursor.execute(f"""SELECT TABLE_NAME FROM LH_SampleData.INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'dbo'""")
                    resultSet = get_result_set(cursor)
                    if len(resultSet) == 14:
                        break
                    else:
                        time.sleep(30) # wait 30 seconds for the schema sync to occur
                cursor.commit()

            for table in resultSet:
                with conn.cursor() as cursor:
                    query = f"""IF OBJECT_ID('dbo.{table.get("TABLE_NAME")}', 'U') IS NOT NULL DROP TABLE dbo.{table.get("TABLE_NAME")}; CREATE TABLE dbo.{table.get("TABLE_NAME")} AS SELECT * FROM LH_SampleData.dbo.{table.get("TABLE_NAME")}"""
                    print(f"{table.get('TABLE_NAME')} - {query}")
                    cursor.execute(query)
                    cursor.commit()
        break
    except:
        if attempt == (20 - 1):
            raise ValueError('Data could not be loaded into Warehouse')
        time.sleep(5)
        continue

In [ ]:
from notebookutils import mssparkutils  
from pyspark.sql import functions as F
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType, TimestampType, DoubleType
import pyodbc, struct, itertools, time, datetime, re, uuid, json

connectionString = f'DRIVER={{ODBC Driver 18 for SQL Server}};SERVER={WH_SampleData.get("properties").get("connectionString")};Database={WH_SampleData.get("displayName")}'

# Use the credentials of the user executing the notebook
token = bytes(mssparkutils.credentials.getToken('pbi'), "UTF-8")
encoded_bytes = bytes(itertools.chain.from_iterable(zip(token, itertools.repeat(0))))
tokenstruct = struct.pack("<i", len(encoded_bytes)) + encoded_bytes

with pyodbc.connect(connectionString, attrs_before = { 1256:tokenstruct }) as conn:
    with conn.cursor() as cursor:
        cursor.execute('''IF OBJECT_ID('dbo.sp_Query', 'P') IS NOT NULL
	DROP PROCEDURE dbo.sp_Query
;''')
        cursor.execute('''CREATE PROCEDURE dbo.sp_Query AS
	SELECT COUNT(*) AS cnt_FactOrder FROM FactOrder

    SELECT COUNT(*) AS cnt_FactMovement FROM FactMovement

    SELECT COUNT(*) AS cnt_FactPurchase FROM FactPurchase

;''')
        cursor.execute('''IF OBJECT_ID('dbo.sp_Ingest', 'P') IS NOT NULL
	DROP PROCEDURE dbo.sp_Ingest
;''')
        cursor.execute('''CREATE PROCEDURE dbo.sp_Ingest AS
        IF OBJECT_ID('dbo.DimCity', 'U') IS NOT NULL DROP TABLE dbo.DimCity; CREATE TABLE dbo.DimCity AS SELECT * FROM LH_SampleData.dbo.DimCity;

        SELECT COUNT(*) FROM DimCity
;''')
        cursor.commit()


In [ ]:
from notebookutils import mssparkutils  
from pyspark.sql import functions as F
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType, TimestampType, DoubleType
import pyodbc, struct, itertools, time, datetime, re, uuid, json

connectionString = f'DRIVER={{ODBC Driver 18 for SQL Server}};SERVER={WH_SampleData.get("properties").get("connectionString")};Database="LH_SampleData"'

# Use the credentials of the user executing the notebook
token = bytes(mssparkutils.credentials.getToken('pbi'), "UTF-8")
encoded_bytes = bytes(itertools.chain.from_iterable(zip(token, itertools.repeat(0))))
tokenstruct = struct.pack("<i", len(encoded_bytes)) + encoded_bytes

with pyodbc.connect(connectionString, attrs_before = { 1256:tokenstruct }) as conn:
    with conn.cursor() as cursor:
        cursor.execute('''IF OBJECT_ID('dbo.sp_Query', 'P') IS NOT NULL
	DROP PROCEDURE dbo.sp_Query
;''')
        cursor.execute('''CREATE PROCEDURE dbo.sp_Query AS
	SELECT COUNT(*) AS cnt_FactOrder FROM FactOrder

    SELECT COUNT(*) AS cnt_FactMovement FROM FactMovement

    SELECT COUNT(*) AS cnt_FactPurchase FROM FactPurchase

;''')
        cursor.commit()


In [ ]:
from notebookutils import mssparkutils  
from pyspark.sql import functions as F
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType, TimestampType, DoubleType
import pyodbc, struct, itertools, time, datetime, re, uuid, json

connectionString = f'DRIVER={{ODBC Driver 18 for SQL Server}};SERVER={WH_SampleData.get("properties").get("connectionString")};Database="LH_QueryResults"'

# Use the credentials of the user executing the notebook
token = bytes(mssparkutils.credentials.getToken('pbi'), "UTF-8")
encoded_bytes = bytes(itertools.chain.from_iterable(zip(token, itertools.repeat(0))))
tokenstruct = struct.pack("<i", len(encoded_bytes)) + encoded_bytes

with pyodbc.connect(connectionString, attrs_before = { 1256:tokenstruct }) as conn:
    with conn.cursor() as cursor:
        cursor.execute('''CREATE OR ALTER VIEW vwColdQueryPerformancePerWorker AS 
/*
Get the last execution of each unique query that was executed per worker.
This will give us the cold query performance for each run.
*/
SELECT *
FROM
(
	SELECT	*
			,ROW_NUMBER() OVER (PARTITION BY RunId, WorkerNum, QueryUniqueNum ORDER BY QueryRepeatNum) AS RN
	FROM LH_QueryResults.dbo.queryresults
) AS a
WHERE RN = 1
;''')

        cursor.execute('''CREATE OR ALTER VIEW vwWarmQueryPerformancePerWorker AS 
/*
Get the last execution of each unique query that was executed per worker.
This will give us the warm query performance for each run.
*/
SELECT *
FROM
(
	SELECT	*
			,ROW_NUMBER() OVER (PARTITION BY RunId, WorkerNum, QueryUniqueNum ORDER BY QueryRepeatNum DESC) AS RN
	FROM LH_QueryResults.dbo.queryresults
) AS a
WHERE RN = 1
;''')

        cursor.execute('''CREATE OR ALTER VIEW vwColdQueryPerformancePerRun AS 
/*
Get the last execution of each unique query that was executed per worker.
This will give us the warm query performance for each run.
*/
SELECT RunId, RunName
		,SUM(QueryDurationMS) AS QueryDurationMS
		,SUM(QueryCUSeconds) AS QueryCUSeconds
		,SUM(QueryCostPayGo) AS QueryCostPayGo
		,SUM(QueryCostReserved) AS QueryCostReserved
		,SUM(DataScannedDiskMB) AS DataScannedDiskMB
		,SUM(DataScannedMemoryMB) AS DataScannedMemoryMB
		,SUM(DataScannedRemoteStorageMB) AS DataScannedRemoteStorageMB
		,SUM(AllocatedCpuTimeMS) AS AllocatedCpuTimeMS
FROM
(
	SELECT	RunId, RunName, QueryDurationMS, QueryCUSeconds, QueryCostPayGo, QueryCostReserved, DataScannedDiskMB
		    ,DataScannedMemoryMB, DataScannedRemoteStorageMB, AllocatedCpuTimeMS
			,ROW_NUMBER() OVER (PARTITION BY RunId, WorkerNum, QueryUniqueNum ORDER BY QueryRepeatNum) AS RN
	FROM LH_QueryResults.dbo.queryresults
) AS a
WHERE RN = 1
GROUP BY RunId, RunName
;''')
        cursor.execute('''CREATE OR ALTER VIEW vwWarmQueryPerformancePerRun AS 
/*
Get the last execution of each unique query that was executed per worker.
This will give us the warm query performance for each run.
*/
SELECT RunId, RunName
		,SUM(QueryDurationMS) AS QueryDurationMS
		,SUM(QueryCUSeconds) AS QueryCUSeconds
		,SUM(QueryCostPayGo) AS QueryCostPayGo
		,SUM(QueryCostReserved) AS QueryCostReserved
		,SUM(DataScannedDiskMB) AS DataScannedDiskMB
		,SUM(DataScannedMemoryMB) AS DataScannedMemoryMB
		,SUM(DataScannedRemoteStorageMB) AS DataScannedRemoteStorageMB
		,SUM(AllocatedCpuTimeMS) AS AllocatedCpuTimeMS
FROM
(
	SELECT	RunId, RunName, QueryDurationMS, QueryCUSeconds, QueryCostPayGo, QueryCostReserved, DataScannedDiskMB
		    ,DataScannedMemoryMB, DataScannedRemoteStorageMB, AllocatedCpuTimeMS
			,ROW_NUMBER() OVER (PARTITION BY RunId, WorkerNum, QueryUniqueNum ORDER BY QueryRepeatNum DESC) AS RN
	FROM LH_QueryResults.dbo.queryresults
) AS a
WHERE RN = 1
GROUP BY RunId, RunName
;''')
        cursor.commit()

#### Refresh Fabric Capacity Metrics App so that the latest workspace artifacts are in the model

In [ ]:
import requests, json, time

def model_refresh():
    header = {'Authorization': f'Bearer {mssparkutils.credentials.getToken("pbi")}'
            ,"Content-Type": "application/json"
            }

    response = requests.get('https://api.fabric.microsoft.com/v1/workspaces', headers=header)

    capacityWorkspaceId = [workspace.get('id') for workspace in response.json().get('value') if workspace.get('displayName') == CapacityMetricsWorkspace][0]
    print(f'{capacityWorkspaceId = }')

    response = requests.get(f"https://api.powerbi.com/v1.0/myorg/groups/{capacityWorkspaceId}/datasets", headers=header)

    datasetId = [dataset.get('id') for dataset in response.json().get('value') if dataset.get('name') == CapacityMetricsDataset][0]
    print(f'{datasetId = }')

    response = requests.post(f"https://api.powerbi.com/v1.0/myorg/groups/{capacityWorkspaceId}/datasets/{datasetId}/refreshes", headers=header)

    refreshId = response.headers.get('RequestId')
    print(f'{refreshId = } | {response.status_code = }')

    if response.status_code == 202:
        for attempt in range(12): 
            # https://learn.microsoft.com/en-us/power-bi/connect-data/asynchronous-refresh#get-refreshes
            response = requests.get(f"https://api.powerbi.com/v1.0/myorg/groups/{capacityWorkspaceId}/datasets/{datasetId}/refreshes?$top=1", headers=header)
            if response.status_code == 200:
                if response.json().get('value')[0].get('status') != 'Unknown':
                    print(f'Refresh Complete')
                    break
                else:
                    print(f'Refreshing tables ...')
                    time.sleep(20)
            else:
                time.sleep(10)
    else:
        print(f'Refreshed failed - {response.text}')

In [ ]:

import requests, math

def check_if_workspace_exists():
  header = {'Authorization': f'Bearer {mssparkutils.credentials.getToken("pbi")}'
            ,"Content-Type": "application/json"
            }

  response = requests.get('https://api.fabric.microsoft.com/v1/workspaces', headers=header)

  capacityWorkspaceId = [workspace.get('id') for workspace in response.json().get('value') if workspace.get('displayName') == CapacityMetricsWorkspace][0]
  print(f'{capacityWorkspaceId = }')

  response = requests.get(f"https://api.powerbi.com/v1.0/myorg/groups/{capacityWorkspaceId}/datasets", headers=header)

  datasetId = [dataset.get('id') for dataset in response.json().get('value') if dataset.get('name') == CapacityMetricsDataset][0]
  print(f'{datasetId = }')

  body = {
    "queries": [
      {
        "query": f"""
            DEFINE
              VAR __DS0Core = 
                DISTINCT('Items'[WorkspaceName])
                
            EVALUATE
              __DS0Core
      """
      }
    ]
  }

  response = requests.post(f'https://api.powerbi.com/v1.0/myorg/datasets/{datasetId}/executeQueries', headers=header, json=body )

  return response

workspaceFound = False
response = check_if_workspace_exists()
for results in response.json().get('results'):
  for table in results.get('tables'):
    for row in table.get('rows'):
      if row.get('Items[WorkspaceName]') == workspaceName:
        print(f'Workspace "{workspaceName}" was found in the capacity metrics app model')
        workspaceFound = True

if not workspaceFound:
  for retry in range(10):
      model_refresh()
      response = check_if_workspace_exists()
      for results in response.json().get('results'):
          for table in results.get('tables'):
              for row in table.get('rows'):
                  if row.get('Items[WorkspaceName]') == workspaceName:
                      workspaceFound = True
      if workspaceFound:
          print(f'Workspace "{workspaceName}" was found in the capacity metrics app model after doing a model refresh.')
          break
      else:
          print(f'Workspace "{workspaceName}" was not found. Waiting 2 minutes and will refresh the model and try again...')
          time.sleep(120)

#### Run the *NB_Query_Cost_Analyzer* notebook. This will execute a sample set of queries against the WH_SampleData and LH_SampleData to demonstrate the functionality.

In [ ]:
import requests, json, time

queryList = [
        # Transaction
        'SELECT COUNT(*) FROM FactTransaction'
        ,'''SELECT	COUNT(*)
        FROM	FactTransaction AS ft
        JOIN	DimDate AS d
        ON		d.Date = ft.DateKey
        JOIN	DimPaymentMethod AS pm
        ON		pm.PaymentMethodKey= ft.PaymentMethodKey
        JOIN	DimTransactionType AS tt
        ON		tt.TransactionTypeKey = ft.TransactionTypeKey
        JOIN	DimSupplier AS s
        ON		s.SupplierKey = ft.SupplierKey
        JOIN	DimCustomer AS cu
        ON		cu.CustomerKey = ft.CustomerKey
        JOIN	DimCustomer AS cuBill
        ON		cuBill.CustomerKey = ft.BillToCustomerKey'''
        # Order
        ,'SELECT COUNT(*) cnt_FactOrder FROM FactOrder'
        ,'''SELECT	COUNT(*)
        FROM	FactOrder AS fo
        JOIN	DimDate AS dOrder
        ON		dOrder.Date = fo.OrderDateKey
        LEFT JOIN	DimDate AS dPicked
        ON		dPicked.Date = fo.PickedDateKey
        JOIN	DimStockItem AS si
        ON		si.StockItemKey = fo.StockItemKey
        JOIN	DimCity AS c
        ON		c.CityKey = fo.CityKey
        JOIN	DimEmployee AS e
        ON		e.EmployeeKey = fo.SalespersonKey
        JOIN	DimEmployee AS ePicker
        ON		ePicker.EmployeeKey = fo.PickerKey
        JOIN	DimCustomer AS cu
        ON		cu.CustomerKey = fo.CustomerKey'''
        # FactMovement
        ,'SELECT COUNT(*) cnt_FactMovement FROM FactMovement'
        ,'''SELECT	COUNT(*)
        FROM	FactMovement AS fm
        JOIN	DimDate AS d
        ON		d.Date = fm.DateKey
        JOIN	DimStockItem AS si
        ON		si.StockItemKey = fm.StockItemKey
        JOIN	DimTransactionType AS tt
        ON		tt.TransactionTypeKey = fm.TransactionTypeKey
        JOIN	DimSupplier AS s
        ON		s.SupplierKey = fm.SupplierKey
        JOIN	DimCustomer AS c
        ON		c.CustomerKey = fm.CustomerKey'''
        # FactPurchase
        ,'SELECT COUNT(*) cnt_FactPurchase FROM FactPurchase'
        ,'''SELECT	COUNT(*)
        FROM	FactPurchase AS fp
        JOIN	DimDate AS d
        ON		d.Date = fp.DateKey
        JOIN	DimStockItem AS si
        ON		si.StockItemKey = fp.StockItemKey
        JOIN	DimSupplier AS s
        ON		s.SupplierKey = fp.SupplierKey'''
        # StockHolding
        ,'SELECT COUNT(*) AS cnt_FactStockHolding FROM FactStockHolding'
        ,'''SELECT	COUNT(*)
        FROM	FactStockHolding AS fsh
        JOIN	DimStockItem AS si
        ON		si.StockItemKey = fsh.StockItemKey'''
            ,'''SELECT	TOP 100 fsh.*
        FROM	FactStockHolding AS fsh
        JOIN	DimStockItem AS si
        ON		si.StockItemKey = fsh.StockItemKey'''
        ,'''EXEC sp_Query'''
    ]

for artifactType in ['LH_SampleData', 'WH_SampleData']:
    print(f'Running queries for {artifactType}')

    header = {'Authorization': f'Bearer {mssparkutils.credentials.getToken("pbi")}'
          ,"Content-Type": "application/json"
          }
    
    body = {
        "executionData": {
            "parameters": {
                "FabricDWWorkspaceName": {"value": workspaceName, "type": "string"}
                ,"FabricDWName": {"value": artifactType, "type": "string"}
                ,"ConcurrencyNum": {"value": "1", "type": "int"}
                ,"CapacityMetricsWorkspace": {"value": CapacityMetricsWorkspace, "type": "string"}
                ,"CapacityMetricsDataset": {"value": CapacityMetricsDataset, "type": "string"}
                ,"QueryList": {"value": str(queryList), "type": "string"}
            }
        }
    }

    response = requests.request(method='post', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/items/{notebookId}/jobs/instances?jobType=RunNotebook', headers=header, data=json.dumps(body))

    startTime= time.time()
    if response.status_code == 202:
        print('Started Execution of Notebook')
        time.sleep(10) # Sleep for 10 seconds to wait for job to get started.
        while True:
            responseStatus = requests.request(method="get", url=response.headers.get('Location'), headers=header)
            print(f"Status: {responseStatus.json().get('status')} | {int(time.time()-startTime)} seconds {'|' + str(responseStatus.json().get('failureReason')) if responseStatus.json().get('failureReason') != None else ''}", end='\r')
            if responseStatus.json().get('status') in ['Succeeded', 'Failed', 'Completed']:
                print(f"Status: {responseStatus.json().get('status')} | {int(time.time()-startTime)} seconds {'|' + str(responseStatus.json().get('failureReason')) if responseStatus.json().get('failureReason') != None else ''}")
                break
            else:
                time.sleep(60) # Wait 1 minute to check the status of the run
    else:
        print('Error trying to execute')

In [ ]:
display(spark.read.format('delta').load(f'{LH_QueryResults.properties.get("abfsPath")}/Tables/runresults'))

In [ ]:
display(spark.read.format('delta').load(f'{LH_QueryResults.properties.get("abfsPath")}/Tables/queryresults'))

In [ ]:
display(spark.read.format('delta').load(f'{LH_QueryResults.properties.get("abfsPath")}/Tables/queryinsightsresults'))